## Desafio ENACOM - Bootcamp Seletivo Data Analytics 2024

Análise exploratória dos dados do ONS

Desenvolvido por: Gustavo Basílio Lima

Import das bibliotecas utilizadas ao decorrer do notebook

In [2]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

## Leitura dos dados

In [3]:
df_all_columns = pd.read_parquet(r'..\dados\reamostrados\resampled_dask_df_all_data.parquet')
df_all_columns

,nom_subsistema,nom_estado,nom_tipousina,nom_tipocombustivel,nom_usina,val_geracao
din_instante,,,,,,
2000-01,SUDESTE,SAO PAULO,HIDROELÉTRICA,Hidráulica,Caconde,23880.900
2000-01,SUL,PARANA,HIDROELÉTRICA,Hidráulica,Gov. Ney Braga,374162.300
2000-01,SUDESTE,MINAS GERAIS,HIDROELÉTRICA,Hidráulica,Itumbiara,574602.300
2000-01,SUDESTE,MINAS GERAIS,HIDROELÉTRICA,Hidráulica,Sobragi,33558.400
2000-01,SUDESTE,MATO GROSSO DO SUL,TÉRMICA,Gás,Willian Arjona,0.000
...,...,...,...,...,...,...
2021-12,NORDESTE,BAHIA,HIDROELÉTRICA,Hidráulica,Paulo Afonso I,543.451
2021-12,SUDESTE,GOIAS,HIDROELÉTRICA,Hidráulica,Serra do Facão,37375.320
2021-12,SUDESTE,GOIAS,HIDROELÉTRICA,Hidráulica,Verde 8,10548.214


## Seleção de colunas e unidade da geração

1. Selecionando a análise da geração por fonte, as colunas que se referem à localidade e nome da usina são descartadas.
2. A priori, é utilizado como index a coluna de data (no formato DateTime), denominado como "din_instante", já em frequência mensal. 
3. Os valores de geração (em $M W_{med} \cdot h / mês$) são divididos por $10^6$, tendo a partir de então a unidade (em $T W_{med} \cdot h / mês$).

In [4]:
df_dropped_columns = df_all_columns.drop(columns=['nom_subsistema','nom_estado','nom_usina'])
df_dropped_columns.index = pd.to_datetime(df_dropped_columns.index.astype(str))
df_dropped_columns.val_geracao = round(df_dropped_columns.val_geracao/1e6,2)
df_dropped_columns

,nom_tipousina,nom_tipocombustivel,val_geracao
din_instante,,,
2000-01-01,HIDROELÉTRICA,Hidráulica,0.02
2000-01-01,HIDROELÉTRICA,Hidráulica,0.37
2000-01-01,HIDROELÉTRICA,Hidráulica,0.57
2000-01-01,HIDROELÉTRICA,Hidráulica,0.03
2000-01-01,TÉRMICA,Gás,0.00
...,...,...,...
2021-12-01,HIDROELÉTRICA,Hidráulica,0.00
2021-12-01,HIDROELÉTRICA,Hidráulica,0.04
2021-12-01,HIDROELÉTRICA,Hidráulica,0.01


## Valor total da geração

Para uma visão mais ampla dos dados, é calculado o valor total da energia gerada para cada mês durante o período disponível.

In [5]:
df_total_val_geracao = pd.DataFrame(data=df_dropped_columns.val_geracao)
df_total_val_geracao = df_total_val_geracao.resample('M', kind='period').sum()
df_total_val_geracao

,val_geracao
din_instante,
2000-01,29.51
2000-02,28.63
2000-03,30.42
2000-04,29.22
2000-05,30.12
...,...
2021-08,49.08
2021-09,49.35
2021-10,49.03


## Split dos valores das linhas para colunas

Com a finalidade de deixar o dataframe mais segmentado quanto a geração por fonte, é realizando o rearranjo para que os valores da geração por fonte compartilhem do mesmo index.

Para isso, é usado o loop for, percorrendo por todos os tipos de usina.
1. Em um dataframe separado, são extraídos os valores de geração para cada tipo de usina;
2. Em seguida, os valores da geração são somados para obter os montantes mensais gerados;
3. Por fim, é aplicado o .join() para uní-los a partir do mesmo index.
Assegurando o processo utilizado, é criado um coluna de erro absoluto, verificando a diferença entre as gerações segmentadas com a geração total.

In [6]:
df_val_geracao_by_resource = df_total_val_geracao
for type_resource in list(df_dropped_columns.nom_tipousina.unique()):
    df_type_resource = df_dropped_columns[df_dropped_columns.nom_tipousina == type_resource].val_geracao.to_frame()
    df_type_resource_resampled = df_type_resource.resample('M', kind='period').sum()
    df_val_geracao_by_resource = df_val_geracao_by_resource.join(df_type_resource_resampled, rsuffix=str('_'+type_resource).lower())
df_val_geracao_by_resource['erro_absoluto_val_geracao'] = round(abs(df_val_geracao_by_resource.val_geracao - df_val_geracao_by_resource.iloc[:,1:].sum(axis=1)),8)
df_val_geracao_by_resource

,val_geracao,val_geracao_hidroelétrica,val_geracao_térmica,val_geracao_nuclear,val_geracao_eolielétrica,val_geracao_bombeamento,val_geracao_fotovoltaica,erro_absoluto_val_geracao
din_instante,,,,,,,,
2000-01,29.51,27.67,1.42,0.42,NaN,NaN,NaN,0.0
2000-02,28.63,26.84,1.39,0.40,NaN,NaN,NaN,0.0
2000-03,30.42,28.76,1.26,0.40,NaN,NaN,NaN,0.0
2000-04,29.22,27.82,1.04,0.36,NaN,NaN,NaN,0.0
2000-05,30.12,28.80,1.32,0.00,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...
2021-08,49.08,24.35,14.25,1.48,8.36,NaN,0.64,0.0
2021-09,49.35,25.42,13.57,1.45,8.12,NaN,0.79,0.0
2021-10,49.03,25.87,13.70,1.41,7.41,NaN,0.64,0.0


### Describe

São apresentados as estatísticas básicas do dataframe da geração por fonte

In [7]:
df_val_geracao_by_resource.describe()

,val_geracao,val_geracao_hidroelétrica,val_geracao_térmica,val_geracao_nuclear,val_geracao_eolielétrica,val_geracao_bombeamento,val_geracao_fotovoltaica,erro_absoluto_val_geracao
count,264.000000,264.000000,264.000000,264.000000,190.000000,25.0,78.000000,264.0
mean,39.159811,32.164318,4.483106,1.153409,1.783526,0.0,0.255128,0.0
std,6.902910,4.163854,3.466161,0.320709,2.126003,0.0,0.226428,0.0
min,23.660000,20.570000,0.700000,0.000000,0.000000,0.0,0.000000,0.0
25%,33.192500,29.297500,1.595000,1.000000,0.082500,0.0,0.000000,0.0
50%,40.050000,32.265000,2.785000,1.245000,0.455000,0.0,0.245000,0.0
75%,44.577500,34.877500,7.132500,1.390000,3.177500,0.0,0.430000,0.0
max,53.710000,42.660000,14.250000,1.500000,8.360000,0.0,0.790000,0.0


## Segmentação da energia térmica por tipo de combustível

Segmentando a análise por fonte, chama atenção a energia térmica que abrange vários tipos de combustíveis.

In [8]:
df_thermal_resource = df_dropped_columns[df_dropped_columns.nom_tipousina == 'TÉRMICA']
df_thermal_resource

,nom_tipousina,nom_tipocombustivel,val_geracao
din_instante,,,
2000-01-01,TÉRMICA,Gás,0.00
2000-01-01,TÉRMICA,Gás,0.30
2000-01-01,TÉRMICA,Biomassa,0.01
2000-01-01,TÉRMICA,Óleo Combustível,0.09
2000-01-01,TÉRMICA,Carvão,0.23
...,...,...,...
2021-12-01,TÉRMICA,Carvão,0.19
2021-12-01,TÉRMICA,Óleo Diesel,0.01
2021-12-01,TÉRMICA,Biomassa,0.01


In [9]:
list(df_dropped_columns[df_dropped_columns.nom_tipousina == 'TÉRMICA'].nom_tipocombustivel.unique())

['Gás',
 'Biomassa',
 'Óleo Combustível',
 'Carvão',
 'Óleo Diesel',
 'Resíduos Industriais',
 'Outras Multi-Combustível']

### Energia térmica por fonte renovável

In [10]:
df_thermal_renewable = df_thermal_resource[df_thermal_resource.nom_tipocombustivel.isin(['Biomassa'])].val_geracao.to_frame().resample('M', kind='period').sum()
df_thermal_renewable

,val_geracao
din_instante,
2000-01,0.05
2000-02,0.01
2000-03,0.00
2000-04,0.00
2000-05,0.00
...,...
2021-08,1.15
2021-09,1.09
2021-10,0.37


### Energia térmica por fonte não renovável

In [11]:
df_thermal_no_renewable = df_thermal_resource[~df_thermal_resource.nom_tipocombustivel.isin(['Biomassa'])].val_geracao.to_frame().resample('M', kind='period').sum()
df_thermal_no_renewable

,val_geracao
din_instante,
2000-01,1.37
2000-02,1.38
2000-03,1.26
2000-04,1.04
2000-05,1.32
...,...
2021-08,13.10
2021-09,12.48
2021-10,13.33


### Análise de erro do processo

In [12]:
df_thermal_resource_by_fuel = pd.concat([df_thermal_renewable,df_thermal_no_renewable], axis=1)
df_thermal_resource_by_fuel.columns = ['val_geracao_térmica_renovável','val_geracao_térmica_não_renovável']
df_thermal_resource_by_fuel = df_thermal_resource_by_fuel.join(df_val_geracao_by_resource.val_geracao_térmica)
df_thermal_resource_by_fuel['erro_absoluto_val_geracao_térmica'] = round(abs(df_thermal_resource_by_fuel.val_geracao_térmica - df_thermal_resource_by_fuel.iloc[:,0:2].sum(axis=1)),8)
df_thermal_resource_by_fuel

,val_geracao_térmica_renovável,val_geracao_térmica_não_renovável,val_geracao_térmica,erro_absoluto_val_geracao_térmica
din_instante,,,,
2000-01,0.05,1.37,1.42,0.0
2000-02,0.01,1.38,1.39,0.0
2000-03,0.00,1.26,1.26,0.0
2000-04,0.00,1.04,1.04,0.0
2000-05,0.00,1.32,1.32,0.0
...,...,...,...,...
2021-08,1.15,13.10,14.25,0.0
2021-09,1.09,12.48,13.57,0.0
2021-10,0.37,13.33,13.70,0.0


### Describe

São apresentados as estatísticas básicas do dataframe da geração térmica por combustível

In [13]:
df_thermal_resource_by_fuel.describe()

,val_geracao_térmica_renovável,val_geracao_térmica_não_renovável,val_geracao_térmica,erro_absoluto_val_geracao_térmica
count,264.000000,264.000000,264.000000,264.0
mean,0.264167,4.218939,4.483106,0.0
std,0.360379,3.206662,3.466161,0.0
min,0.000000,0.700000,0.700000,0.0
25%,0.000000,1.580000,1.595000,0.0
50%,0.030000,2.785000,2.785000,0.0
75%,0.470000,6.622500,7.132500,0.0
max,1.200000,13.330000,14.250000,0.0


## Junção dos dataframes

Proseguindo a análise por fonte, são encorporandos os dados da geração térmica por tipo de combustível.

In [14]:
df_val_geracao_by_resource = df_val_geracao_by_resource.join(df_thermal_resource_by_fuel.drop(columns=['erro_absoluto_val_geracao_térmica', 'val_geracao_térmica']))
df_val_geracao_by_resource

,val_geracao,val_geracao_hidroelétrica,val_geracao_térmica,val_geracao_nuclear,val_geracao_eolielétrica,val_geracao_bombeamento,val_geracao_fotovoltaica,erro_absoluto_val_geracao,val_geracao_térmica_renovável,val_geracao_térmica_não_renovável
din_instante,,,,,,,,,,
2000-01,29.51,27.67,1.42,0.42,NaN,NaN,NaN,0.0,0.05,1.37
2000-02,28.63,26.84,1.39,0.40,NaN,NaN,NaN,0.0,0.01,1.38
2000-03,30.42,28.76,1.26,0.40,NaN,NaN,NaN,0.0,0.00,1.26
2000-04,29.22,27.82,1.04,0.36,NaN,NaN,NaN,0.0,0.00,1.04
2000-05,30.12,28.80,1.32,0.00,NaN,NaN,NaN,0.0,0.00,1.32
...,...,...,...,...,...,...,...,...,...,...
2021-08,49.08,24.35,14.25,1.48,8.36,NaN,0.64,0.0,1.15,13.10
2021-09,49.35,25.42,13.57,1.45,8.12,NaN,0.79,0.0,1.09,12.48
2021-10,49.03,25.87,13.70,1.41,7.41,NaN,0.64,0.0,0.37,13.33


## Exclusão de colunas e upload dos dados

Pouco agregando para análises desse tipo de cenário, baseado na descrição estatística anterior, é optado para a remoção da coluna "val_geracao_bombeamento".

Além disso, já constatado o erro absoluto, para o prosseguimento da análise, também é removido a coluna "erro_absoluto_val_geracao".

In [15]:
df_val_geracao_by_resource = df_val_geracao_by_resource.drop(columns=['val_geracao_bombeamento', 'erro_absoluto_val_geracao'])
df_val_geracao_by_resource.to_csv(r'..\dados\segmentados\df_val_geracao_by_resource.csv')
df_val_geracao_by_resource

,val_geracao,val_geracao_hidroelétrica,val_geracao_térmica,val_geracao_nuclear,val_geracao_eolielétrica,val_geracao_fotovoltaica,val_geracao_térmica_renovável,val_geracao_térmica_não_renovável
din_instante,,,,,,,,
2000-01,29.51,27.67,1.42,0.42,NaN,NaN,0.05,1.37
2000-02,28.63,26.84,1.39,0.40,NaN,NaN,0.01,1.38
2000-03,30.42,28.76,1.26,0.40,NaN,NaN,0.00,1.26
2000-04,29.22,27.82,1.04,0.36,NaN,NaN,0.00,1.04
2000-05,30.12,28.80,1.32,0.00,NaN,NaN,0.00,1.32
...,...,...,...,...,...,...,...,...
2021-08,49.08,24.35,14.25,1.48,8.36,0.64,1.15,13.10
2021-09,49.35,25.42,13.57,1.45,8.12,0.79,1.09,12.48
2021-10,49.03,25.87,13.70,1.41,7.41,0.64,0.37,13.33


## Correlação entre variáveis

Para observar o efeito de uma variável na outra, é aplicado a correlação de Pearson. Este tipo de correlação calcula a razão entre a covariância de duas variáveis e o produto de seus desvios padrão. Onde o valor 1 aponta que as variáveis têm uma relação diretamente proporcional, enquanto o valor -1, inversamente proporcional.

In [16]:
df_val_geracao_by_resource_to_corr = df_val_geracao_by_resource.drop(columns=['val_geracao'])
df_val_geracao_by_resource_to_corr.corr('pearson').style.background_gradient(cmap='coolwarm')

,val_geracao_hidroelétrica,val_geracao_térmica,val_geracao_nuclear,val_geracao_eolielétrica,val_geracao_fotovoltaica,val_geracao_térmica_renovável,val_geracao_térmica_não_renovável
val_geracao_hidroelétrica,1.000000,0.064740,0.151944,-0.394788,-0.151086,-0.012567,0.071392
val_geracao_térmica,0.064740,1.000000,0.305212,0.629050,0.248228,0.745102,0.997187
val_geracao_nuclear,0.151944,0.305212,1.000000,0.123514,-0.040769,0.239613,0.302982
val_geracao_eolielétrica,-0.394788,0.629050,0.123514,1.000000,0.778561,0.839932,0.577125
val_geracao_fotovoltaica,-0.151086,0.248228,-0.040769,0.778561,1.000000,0.162380,0.233295
val_geracao_térmica_renovável,-0.012567,0.745102,0.239613,0.839932,0.162380,1.000000,0.693014
val_geracao_térmica_não_renovável,0.071392,0.997187,0.302982,0.577125,0.233295,0.693014,1.000000


## Plot da geração

Para melhor visualizar o que os dados nos mostram, é plotado um gráfico onde mostra a geração total de energia elétrica por mês , sendo segmentado pelas fontes de energia.

In [17]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_resource.index.astype(str)), y=df_val_geracao_by_resource.val_geracao,
                         mode='lines', name='Geração Total'))
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_resource.index.astype(str)), y=df_val_geracao_by_resource.val_geracao_hidroelétrica,
                         mode='lines', name='Hidroelétrica'))
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_resource.index.astype(str)), y=df_val_geracao_by_resource.val_geracao_térmica_renovável,
                         mode='lines', name='Térmica Renovável'))
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_resource.index.astype(str)), y=df_val_geracao_by_resource.val_geracao_térmica_não_renovável,
                         mode='lines', name='Térmica Não Renovável'))
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_resource.index.astype(str)), y=df_val_geracao_by_resource.val_geracao_nuclear,
                         mode='lines', name='Nuclear'))
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_resource.index.astype(str)), y=df_val_geracao_by_resource.val_geracao_eolielétrica,
                         mode='lines', name='Eolielétrica'))
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_resource.index.astype(str)), y=df_val_geracao_by_resource.val_geracao_fotovoltaica,
                         mode='lines', name='Fotovoltaica'))

fig.update_layout(title='Geração de Energia Elétrica por Fonte no Brasil (2000-2021)',
                   xaxis_title='Mês e Ano',
                   yaxis_title='Energia Gerada (TWh/mês)')

fig.show()

## Normalização da geração

Complementando a análise de correlação entre as variáveis, o gráfico anterior não traduz a relação entre as variáveis. Sendo assim, é resolvido normalizar os dados, num limiar de 0 a 1, baseado em seus valores máximos e mínimos.

In [18]:
scaler = MinMaxScaler()
scaler.fit(df_val_geracao_by_resource_to_corr)
df_val_geracao_by_resource_to_corr_normalized = scaler.transform(df_val_geracao_by_resource_to_corr)
df_val_geracao_by_resource_to_corr_normalized = pd.DataFrame(data=df_val_geracao_by_resource_to_corr_normalized, columns=df_val_geracao_by_resource_to_corr.columns)
df_val_geracao_by_resource_to_corr_normalized.index = df_val_geracao_by_resource.index
df_val_geracao_by_resource_to_corr_normalized

,val_geracao_hidroelétrica,val_geracao_térmica,val_geracao_nuclear,val_geracao_eolielétrica,val_geracao_fotovoltaica,val_geracao_térmica_renovável,val_geracao_térmica_não_renovável
din_instante,,,,,,,
2000-01,0.321412,0.053137,0.280000,NaN,NaN,0.041667,0.053048
2000-02,0.283839,0.050923,0.266667,NaN,NaN,0.008333,0.053840
2000-03,0.370756,0.041328,0.266667,NaN,NaN,0.000000,0.044339
2000-04,0.328203,0.025092,0.240000,NaN,NaN,0.000000,0.026920
2000-05,0.372567,0.045756,0.000000,NaN,NaN,0.000000,0.049089
...,...,...,...,...,...,...,...
2021-08,0.171118,1.000000,0.986667,1.000000,0.810127,0.958333,0.981789
2021-09,0.219556,0.949815,0.966667,0.971292,1.000000,0.908333,0.932700
2021-10,0.239928,0.959410,0.940000,0.886364,0.810127,0.308333,1.000000


## Plot da geração normalizada

Com esse gráfico, o que fica mais evidenciado é a evolução da geração das fontes, com a busca da diversificação da matriz (tanto por fatores econômicos, como por fatores climáticos). Também é possível visualizar os efeitos de complementaridade entre as fontes, como também os efeitos de sazonalidades.

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_resource.index.astype(str)), y=df_val_geracao_by_resource_to_corr_normalized.val_geracao_hidroelétrica,
                         mode='lines', name='Hidroelétrica'))
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_resource.index.astype(str)), y=df_val_geracao_by_resource_to_corr_normalized.val_geracao_térmica_renovável,
                         mode='lines', name='Térmica Renovável'))
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_resource.index.astype(str)), y=df_val_geracao_by_resource_to_corr_normalized.val_geracao_térmica_não_renovável,
                         mode='lines', name='Térmica Não Renovável'))
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_resource.index.astype(str)), y=df_val_geracao_by_resource_to_corr_normalized.val_geracao_nuclear,
                         mode='lines', name='Nuclear'))
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_resource.index.astype(str)), y=df_val_geracao_by_resource_to_corr_normalized.val_geracao_eolielétrica,
                         mode='lines', name='Eolielétrica'))
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_resource.index.astype(str)), y=df_val_geracao_by_resource_to_corr_normalized.val_geracao_fotovoltaica,
                         mode='lines', name='Fotovoltaica'))

fig.update_layout(title='Geração de Energia Elétrica Unitária por Fonte no Brasil (2000-2021)',
                   xaxis_title='Mês e Ano',
                   yaxis_title='Energia Gerada (unitário)')

fig.show()

## Seleção de variáveis para os modelos preditivos

Dividir a análise exploratória da geração de energia elétrica no Brasil em dois grupos (fontes renováveis e não renováveis) simplifica a avaliação, destacando o impacto ambiental. A categorização permite uma visão clara das implicações de cada grupo, facilitando a identificação de padrões, avaliação de sustentabilidade a longo prazo e elaboração de estratégias de transição energética.

Para os dados obitidos, as fontes renováveis são hidroelétrica, térmica renovável, eólica e fotovoltaica, já as não renováveis são térmica não renovável e nuclear. Com isso, é possível observar o perfil de geração para esses segmentos.


In [20]:
df_val_geracao_by_macro_resource = pd.DataFrame(index=df_val_geracao_by_resource.index)
df_val_geracao_by_macro_resource['val_geracao_renovavel'] = df_val_geracao_by_resource[['val_geracao_hidroelétrica',
                                                                                        'val_geracao_térmica_renovável',
                                                                                        'val_geracao_eolielétrica',
                                                                                        'val_geracao_fotovoltaica']].sum(axis=1)
df_val_geracao_by_macro_resource['val_geracao_nao_renovavel'] = df_val_geracao_by_resource[['val_geracao_térmica_não_renovável',
                                                                                            'val_geracao_nuclear']].sum(axis=1)
df_val_geracao_by_macro_resource.to_csv(r'..\dados\segmentados\df_val_geracao_by_macro_resource.csv')
df_val_geracao_by_macro_resource

,val_geracao_renovavel,val_geracao_nao_renovavel
din_instante,,
2000-01,27.72,1.79
2000-02,26.85,1.78
2000-03,28.76,1.66
2000-04,27.82,1.40
2000-05,28.80,1.32
...,...,...
2021-08,34.50,14.58
2021-09,35.42,13.93
2021-10,34.29,14.74


### Plot da geração renovável x não renovável

In [21]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_macro_resource.index.astype(str)), y=df_val_geracao_by_macro_resource.val_geracao_renovavel	,
                         mode='lines', name='Fontes Renováveis'))
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_macro_resource.index.astype(str)), y=df_val_geracao_by_macro_resource.val_geracao_nao_renovavel	,
                         mode='lines', name='Fontes Não Renováveis'))

fig.update_layout(title='Geração de Energia Elétrica por Fonte no Brasil (2000-2021)',
                   xaxis_title='Mês e Ano',
                   yaxis_title='Energia Gerada ((TWh/mês))')

fig.show()

### Geração anual de energia por tipo de fonte

In [33]:
df_val_geracao_by_macro_resource_per_year = df_val_geracao_by_macro_resource.resample('Y', kind='period').sum()
df_val_geracao_by_macro_resource_per_year

,val_geracao_renovavel,val_geracao_nao_renovavel
din_instante,,
2000,335.70,20.85
2001,292.30,33.81
2002,315.47,31.09
2003,336.77,28.59
2004,351.94,33.11
2005,370.38,30.26
2006,382.38,33.77
2007,406.66,30.91
2008,398.32,50.23


In [35]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_macro_resource_per_year.index.astype(str)).year,
                         y=df_val_geracao_by_macro_resource_per_year.val_geracao_renovavel,
                         mode='lines', name='Fontes Renováveis'))
fig.add_trace(go.Scatter(x=pd.to_datetime(df_val_geracao_by_macro_resource_per_year.index.astype(str)).year,
                         y=df_val_geracao_by_macro_resource_per_year.val_geracao_nao_renovavel,
                         mode='lines', name='Fontes Não Renováveis'))

fig.update_layout(title='Geração de Energia Elétrica por Fonte no Brasil (2000-2021)',
                   xaxis_title='Ano',
                   yaxis_title='Energia Gerada (TWh/ano)')

fig.show()